Libraries

In [92]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

Read Data

In [93]:
# Path to the dataset file
path = r"C:\Users\mrtat\Downloads\NTI\Data\dataEDA.csv" 
df = pd.read_csv(path, encoding='latin1') # Load the dataset
# Convert 'InvoiceDate' to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')
df.head(5) # Display the first 5 rows of the dataset

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [94]:
df.info() # Display information about the dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


Preprocessing

In [95]:
print("NULL values in each column:",df.isnull().sum().sum()) # Check for NULL values in the dataset
print("Duplicate rows:", df.duplicated().sum()) # Check for duplicate rows in the dataset

NULL values in each column: 136534
Duplicate rows: 5268


In [96]:
cols = ['Quantity', 'UnitPrice']
df['Quantity']= df['Quantity'].astype(int) # Convert 'Quantity' to integer type
for col in cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')  # Ensure numeric
    df[col] = np.where(df[col] > 0,df[col], np.nan)          # Replace 0 or negative with NaN
df = df[df['Quantity'].notna()] 

In [97]:
df.dropna(subset=['CustomerID'], inplace=True)

# Fill missing values
for col in df.columns : 
    if col == 'Country':
         df[col] = df[col].fillna(df.groupby('CustomerID')[col].transform(
            lambda x:x.mode()[0] 
        ))
         df[col] = df.groupby('CustomerID')[col].transform(
            lambda x:x.mode()[0] 
        )
    elif col == 'InvoiceDate':   
        df[col] = df[col].fillna(df.groupby('InvoiceNo')[col].transform(
            lambda x:x.mode()[0] 
        ))
        df[col] = df.groupby('InvoiceNo')[col].transform(
            lambda x:x.mode()[0] 
        )
    elif col == 'UnitPrice': 
         
        df[col] = df[col].fillna(df.groupby('StockCode')[col].transform(
           lambda x: x.mode()[0] if not x.mode().empty else x.median()
        ))
        df[col] = df.groupby('StockCode')[col].transform(
           lambda x: x.mode()[0] if not x.mode().empty else x.median()
        )
        df['UnitPrice'] = df['UnitPrice'].round(
        decimals=2)  # Round the UnitPrice to 2 decimal places
        df[col] = np.where(df[col] > 0,df[col], np.nan)
    
    df = df[df['UnitPrice'].notna()]  # Drop rows where UnitPrice is NaN

In [98]:
df.drop_duplicates(inplace=True) # Remove duplicate rows
df.reset_index(drop=True, inplace=True) # Reset index after dropping duplicates

In [99]:
print("NULL values in each column:\n",df.isnull().sum().sort_values(ascending=False)) # Check for NULL values in the dataset
print("Duplicate rows:", df.duplicated().sum()) # Check for duplicate rows in the dataset

NULL values in each column:
 InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64
Duplicate rows: 0


In [100]:
df.drop(['CustomerID'],axis=1 ,inplace=True) # Drop 'CustomerID' 

In [101]:
df['TotalPrice'] = df['Quantity'] * df['UnitPrice'] # Create a new column 'TotalPrice'

In [102]:
desc=df.drop(columns=['InvoiceNo', 'StockCode', 'Description', 'Country', 'InvoiceDate'], axis=1) # Drop non-numeric columns for description
desc.describe() # Display descriptive statistics of the dataset

,Quantity,UnitPrice,TotalPrice
count,392668.000000,392668.000000,392668.000000
mean,13.120043,2.935146,24.195487
std,180.498205,3.980273,318.695755
min,1.000000,0.040000,0.060000
25%,2.000000,1.250000,4.800000
50%,6.000000,1.950000,11.900000
75%,12.000000,3.750000,19.800000
max,80995.000000,165.000000,168469.600000


In [103]:
df['Quantity'].sort_values(ascending=False).head(10) # Display the top 10 highest quantities

392202    80995.0
36516     74215.0
304632     4800.0
141547     4300.0
60682      3906.0
186653     3186.0
32194      3114.0
108193     3114.0
314486     3000.0
201774     2880.0
Name: Quantity, dtype: float64

In [104]:
print("Total revenue = ",df['TotalPrice'].sum().round(
        decimals=2))  # Round the UnitPrice to 2 decimal places) # Calculate the total price of all transactions

Total revenue =  9500793.38


In [105]:
print("Number of transactions.",df['InvoiceNo'].nunique()) # Display the number of unique transactions

Number of transactions. 18532
